<a href="https://colab.research.google.com/github/jmlDC/MediaBias-Thesis22-23/blob/Modeling/bi_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

reference
https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/nlp/ipynb/bidirectional_lstm_imdb.ipynb#scrollTo=6Rf3oHV3zOzD 

*   https://towardsdatascience.com/multiclass-text-classification-using-lstm-in-pytorch-eac56baed8df
*   https://towardsdatascience.com/sentiment-analysis-using-lstm-and-glove-embeddings-99223a87fe8e



## Installs

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding

In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.4 MB/s eta 0:00:00


## Gdrive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

dir  = "/content/gdrive/MyDrive/THESIS-MS/Git-Thesis22-23/"

Mounted at /content/gdrive


## Dataset

In [4]:
max_features = 20000  # Only consider the top 20k words
maxLen = 500  # Only consider the first 200 words of each movie review

In [5]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv(f'{dir}Official/MFC_prepared.csv', usecols=["code_frames", "annotations"], header=0)
df.code_frames = df.code_frames.astype(int)

label_dict = {}
for x in range(15):
    label_dict[x+1] = x

label_dict
df['label'] = df.code_frames.replace(label_dict)
df

,code_frames,annotations,label
0,10,Immigrants without HOPE need help entering col...,9
1,5,"But in the eyes of the law, he is an illegal i...",4
2,15,"Reaction to Tancredo, Lamm as predicted",14
3,13,"That, said the congressman, is what always hap...",12
4,1,"$50,000 per entry",0
...,...,...,...
46794,11,Smoking is becoming a social taboo,10
46795,5,Nor does it aid lawyers seeking novel ways to...,4
46796,15,'Ashes to Ashes',14
46797,15,SMOKE SCREEN IS SEEN BEHIND THE SMOKELESS,14


## Prep

### Tain and Test Splitting

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(df.annotations.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42,
                                                  stratify=df.label.values
                                                )

In [7]:
X_train.shape

(39779,)

### tokenizing

In [8]:
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

In [9]:
words_to_index = tokenizer.word_index
words_to_index

{'the': 1,
 'to': 2,
 'of': 3,
 'a': 4,
 'in': 5,
 'and': 6,
 'for': 7,
 'that': 8,
 'on': 9,
 'is': 10,
 'gun': 11,
 'was': 12,
 'said': 13,
 'with': 14,
 'by': 15,
 'as': 16,
 'it': 17,
 'marriage': 18,
 'he': 19,
 'be': 20,
 'from': 21,
 'at': 22,
 'court': 23,
 'have': 24,
 'are': 25,
 'who': 26,
 'not': 27,
 'his': 28,
 'gay': 29,
 'state': 30,
 'an': 31,
 'death': 32,
 'has': 33,
 'same': 34,
 'would': 35,
 'but': 36,
 'they': 37,
 'new': 38,
 'law': 39,
 'sex': 40,
 'will': 41,
 'this': 42,
 'their': 43,
 'bill': 44,
 'tobacco': 45,
 'ban': 46,
 'more': 47,
 'or': 48,
 'guns': 49,
 'immigration': 50,
 'about': 51,
 'one': 52,
 'had': 53,
 'after': 54,
 's': 55,
 'control': 56,
 'year': 57,
 'federal': 58,
 'were': 59,
 'smoking': 60,
 'u': 61,
 'two': 62,
 'states': 63,
 'been': 64,
 'people': 65,
 'which': 66,
 'no': 67,
 'than': 68,
 'i': 69,
 'penalty': 70,
 'up': 71,
 'case': 72,
 'house': 73,
 'supreme': 74,
 'other': 75,
 'rights': 76,
 'out': 77,
 'all': 78,
 'last': 79,


### GLOVE

In [10]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

word_to_vec_map = read_glove_vector(f'{dir}glove.6B/glove.6B.100d.txt')

In [11]:
# added plus 1
vocab_len = len(words_to_index) +1
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
#   print(embedding_vector)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)


## Model

In [12]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
embeddings = embedding_layer(inputs)                    #added this for glove
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 128)         2560000   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 2,757,761
Trainable params: 2,757,761
Non-train

In [13]:
X_train_indices = tokenizer.texts_to_sequences(X_train)
X_val_indices = tokenizer.texts_to_sequences(X_val)

x_train = keras.utils.pad_sequences(X_train_indices, maxlen=maxLen)
x_val = keras.utils.pad_sequences(X_val_indices, maxlen=maxLen)


In [14]:
x_train.shape,y_train.shape

((39779, 500), (39779,))

In [15]:
x_val.shape, y_val.shape

((7020, 500), (7020,))

In [ ]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_val, y_val))


Epoch 1/10
 106/1244 [=>............................] - ETA: 4:18 - loss: -105.8984 - accuracy: 0.0062

In [ ]:
# (x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(
#     num_words=max_features
# )
# print(len(x_train), "Training sequences")
# print(len(x_val), "Validation sequences")
# x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
# x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)


In [ ]:
# model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
# model.fit(x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val))
